## 뉴스 Crawling ㄱㄱ

In [1]:
# 일단 다 긁어온거라 추후에 필요없는건 삭제 필요요

from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time
from datetime import datetime
import math


# --------------------------------------
# 웹페이지 열고 이동하기 
# --------------------------------------
driver = Chrome()

# 브라우저 크기 최대화
driver.maximize_window()

In [11]:
# --------------------------------------
# 종목 코드 이용하여 주식 뉴스 페이지 들어가기
# --------------------------------------
stockCode = '005930' 	# 삼성전자 종목 코드
startPage = 1			# 뉴스 수집 시작 페이지
lastPage = 10 			# 뉴스 수집 마지막 페이지

In [12]:
# news_data 초기화
news_data = []

# 시작 페이지부터 마지막 페이지까지 순서대로 뉴스 데이터 수집집
for pageNum in range(startPage, lastPage + 1):
	url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
	driver.get(url)

	# iframe이 로드될 때까지 대기
	try: 
		WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "news_frame")))

		# iframe으로 전환 (name 또는 id 사용 가능)
		driver.switch_to.frame("news_frame") # 여기선 id 사용

		# 모든 tr 선택
		rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

		for i, row in enumerate(rows):
			# tr의 class 속성 가져오기
			tr_class = row.get_attribute("class") or ""
			if "relation_lst" in tr_class:
				continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨
			
			# 메인 기사만 추출
			tds = row.find_elements(By.TAG_NAME, "td")
			if len(tds) >= 3:
				try:
					a_tag = tds[0].find_element(By.TAG_NAME, "a")
					newsTitle = a_tag.text
					newsAddress = a_tag.get_attribute("href")
				except:
					newsTitle = ""
					newsAddress = ""
				
				if newsTitle.strip():
					newsFrom = tds[1].text
					newsDate = tds[2].text
					#print(newsTitle, newsFrom, newsDate)
					news_data.append({'title': newsTitle, 'from': newsFrom, 'date': newsDate, 'link' : newsAddress})

		print(f"페이지 {pageNum} 완료 - 현재까지 {len(news_data)}개 뉴스")

		
	except Exception as e:
		print(f"페이지 {pageNum}에서 오류 발생: {e}")

	finally:
        # 다음 페이지를 위해 메인 프레임으로 복귀 
		driver.switch_to.default_content()

df = pd.DataFrame(news_data)
df

페이지 1 완료 - 현재까지 11개 뉴스
페이지 2 완료 - 현재까지 24개 뉴스
페이지 3 완료 - 현재까지 35개 뉴스
페이지 4 완료 - 현재까지 47개 뉴스
페이지 5 완료 - 현재까지 60개 뉴스
페이지 6 완료 - 현재까지 71개 뉴스
페이지 7 완료 - 현재까지 82개 뉴스
페이지 8 완료 - 현재까지 92개 뉴스
페이지 9 완료 - 현재까지 104개 뉴스
페이지 10 완료 - 현재까지 122개 뉴스


,title,from,date,link
0,"이재용 회장, 2025 삼성 호암상 시상식 참석",연합뉴스,2025.05.30 15:55,https://n.news.naver.com/mnews/article/001/001...
1,삼성 호암상 시상식 참석하는 이재용 회장과 백수현 사장,연합뉴스,2025.05.30 15:53,https://n.news.naver.com/mnews/article/001/001...
2,"노태문 사장, 삼성호암상 시상식 참석",뉴스1,2025.05.30 15:53,https://n.news.naver.com/mnews/article/421/000...
3,금리·절세에 환차익까지 美채권 은근히 짭짤하네,매일경제,2025.05.30 15:52,https://n.news.naver.com/mnews/article/009/000...
4,호암상 시상식 참석하는 이재용 회장,뉴시스,2025.05.30 15:51,https://n.news.naver.com/mnews/article/003/001...
...,...,...,...,...
117,블룸버그 “애플도 9월부터 OS 명칭에 연도 붙인다”,조선비즈,2025.05.29 18:39,https://n.news.naver.com/mnews/article/366/000...
118,"삼성·MS처럼…""애플도 OS 명칭에 연도 붙인다""",연합뉴스,2025.05.29 15:22,https://n.news.naver.com/mnews/article/001/001...
119,"'밸류업 1년' 자사주 소각 활발... 상장사, 5개월만에 21조 태웠...",파이낸셜뉴스,2025.05.29 18:25,https://n.news.naver.com/mnews/article/014/000...
120,엔비디아 깜짝 실적에 반도체株 강세… 상승 탄력 붙을까,파이낸셜뉴스,2025.05.29 18:16,https://n.news.naver.com/mnews/article/014/000...


In [13]:
# DataFrame을 csv 파일로 출력력

uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_news.csv'
#file_path_excel = f'data/{uniqueTime}_news.xlsx'

df.to_csv(file_path_csv, index = False, encoding = "utf-8-sig")
print(f"{startPage}페이지부터 {lastPage}페이지까지 총 {lastPage - startPage + 1}페이지에서 {(len(df))}개의의 뉴스 기사를 CSV 파일({file_path_csv})로 저장하였습니다.")

1페이지부터 10페이지까지 총 10페이지에서 122개의의 뉴스 기사를 CSV 파일(data/20250530_155820_news.csv)로 저장하였습니다.


In [ ]:
### 여기서부턴 초기버전 코드. 필요 X 

In [ ]:
url = f'https://finance.naver.com/item/news.naver?code={stockCode}&page={pageNum}'
driver.get(url)

In [ ]:
# iframe이 로드될 때까지 대기
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, "news_frame"))
)

In [ ]:
# iframe으로 전환 (name 또는 id 사용 가능)
driver.switch_to.frame("news_frame")  


In [ ]:
# 모든 tr 선택
rows = driver.find_elements(By.CSS_SELECTOR, "table.type5 > tbody > tr")

for i, row in enumerate(rows):
    # tr의 class 속성 가져오기
    tr_class = row.get_attribute("class")
    if tr_class and "relation_lst_cluster" in tr_class:
        continue  # relation_lst가 포함된 tr은 건너뜀  ---> 제대로 작동 안 됨

    tds = row.find_elements(By.TAG_NAME, "td")
    if len(tds) >= 3:
        try:
            newsTitle = tds[0].find_element(By.TAG_NAME, "a").text
        except:
            newsTitle = ""
        newsFrom = tds[1].text
        newsDate = tds[2].text
        print(newsTitle, newsFrom, newsDate)
        news_data.append({'title': newsTitle, 'from': newsFrom, 'date': newsDate})


news_data

In [ ]:
df = pd.DataFrame(news_data)
df